In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("data/data.csv")

In [3]:
df.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [4]:
df.groupby(df["deceptive"]).count()

,hotel,polarity,source,text
deceptive,,,,
deceptive,800,800,800,800
truthful,800,800,800,800


In [5]:
#Extracting only the requireed features
df1 = df[['deceptive', 'text']]
df1

,deceptive,text
0,truthful,We stayed for a one night getaway with family ...
1,truthful,Triple A rate with upgrade to view room was le...
2,truthful,This comes a little late as I'm finally catchi...
3,truthful,The Omni Chicago really delivers on all fronts...
4,truthful,I asked for a high floor away from the elevato...
...,...,...
1595,deceptive,Problems started when I booked the InterContin...
1596,deceptive,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,"The Palmer House Hilton, while it looks good i..."


In [6]:
df1.loc[df1['deceptive'] == 'deceptive', 'deceptive'] = 0
df1.loc[df1['deceptive'] == 'truthful', 'deceptive'] = 1

C:\Users\bhanu prakash\anaconda3\envs\project\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\bhanu prakash\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\bhanu prakash\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [7]:
df1.head()

,deceptive,text
0,1,We stayed for a one night getaway with family ...
1,1,Triple A rate with upgrade to view room was le...
2,1,This comes a little late as I'm finally catchi...
3,1,The Omni Chicago really delivers on all fronts...
4,1,I asked for a high floor away from the elevato...


In [8]:
df1.tail()

,deceptive,text
1595,0,Problems started when I booked the InterContin...
1596,0,The Amalfi Hotel has a beautiful website and i...
1597,0,The Intercontinental Chicago Magnificent Mile ...
1598,0,"The Palmer House Hilton, while it looks good i..."
1599,0,"As a former Chicagoan, I'm appalled at the Ama..."


In [9]:
df1.shape

(1600, 2)

In [24]:
df1.isna().sum()

deceptive    0
text         0
dtype: int64

In [32]:
df1['deceptive']=df1['deceptive'].astype(int)

C:\Users\bhanu prakash\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [34]:
corpus = []
for i in range(0, 1600):
    tweet = re.sub('[^a-zA-Z]', ' ', df1['text'][i])
    tweet = tweet.lower()
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
cv = CountVectorizer(max_features=100)

In [36]:
x = cv.fit_transform(corpus).toarray()
y = df1["deceptive"].values

In [37]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

C:\Users\bhanu prakash\anaconda3\envs\project\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [46]:
classifier.score(X_test,y_test)

0.69375

In [47]:
classifier.score(X_train,y_train)

0.9910714285714286

In [48]:
from sklearn.linear_model import LogisticRegression
logreg =LogisticRegression(class_weight='balanced')
sd=logreg.fit(X_train,y_train)
logreg.score(X_train,y_train)

C:\Users\bhanu prakash\anaconda3\envs\project\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8276785714285714

In [49]:
logreg.score(X_test,y_test)

0.7791666666666667

In [50]:

from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear')
model = clf.fit(X_train, y_train)
model.score(X_train, y_train)

0.8321428571428572

In [51]:
model.score(X_test,y_test)

0.7770833333333333

In [71]:
y_pred = model.predict(X_test)

In [73]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test,y_pred)

array([[187,  58],
       [ 49, 186]], dtype=int64)

In [74]:
y_pred

array([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,

In [52]:
from sklearn.tree import DecisionTreeClassifier
c=DecisionTreeClassifier().fit(X_train,y_train)
#accuracy on Training dataset
c.score(X_train,y_train)

1.0

In [53]:
c.score(X_test,y_test)

0.6583333333333333

In [54]:
from sklearn.naive_bayes import MultinomialNB
np = MultinomialNB()
np.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
np.score(X_train,y_train)

0.8035714285714286

In [58]:
np.score(X_test,y_test)

0.7604166666666666

In [80]:
import pickle
pickle.dump(cv, open('tranform.pkl', 'wb'))
pickle.dump(model, open("algo.pk", 'wb'))

In [81]:
clf = pickle.load(open("algo.pk", "rb"))
cv1 = pickle.load(open("tranform.pkl", "rb"))

In [82]:
data = ['We stayed for a one night getaway with family on a thursday. Triple AAA rate of 173 was a steal. 7th floor room complete with 44in plasma TV bose stereo, voss and evian water, and gorgeous bathroom(no tub but was fine for us) Concierge was very helpful. You cannot beat this location... Only flaw was breakfast was pricey and service was very very slow(2hours for four kids and four adults on a friday morning) even though there were only two other tables in the restaurant. Food was very good so it was worth the wait. I would return in a heartbeat. A gem in chicago..']

In [83]:
vect = cv1.transform(data).toarray()
print(vect)
print(clf.predict(vect))

[[0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1]]
[1]


In [84]:
data1 = ["As a former Chicagoan, I'm appalled at the Amalfi Hotel Chicago. First of all, I was expecting luxury and hospitality, neither of which I received. There's an Experience Designer who is supposed to be like a 'personal concierge,' but my experience with my ED was terrible. I felt like he was trying to pressure me into staying more days than I wanted to. Not only that, but I couldn't understand what he was saying most of the time because he was talking so fast. When I finally got to my room, I was disappointed with the quality of the furniture and the room's cleanliness. I had to ask for a maid to come and give me clean towels because some of the towels in the bathroom were damp. On top of that, the bed was messily done; I could have done a better job on my own bed at home. I was angry at this point, because I was paying a lot of money for every night I was staying at Amalfi, and I didn't expect to be greeted with wet towels. I needed to use the Wi-Fi to download some important documents, and the internet was surprisingly slow. Even a very basic hotel or motel could have offered better, maybe even faster internet access. When I finally checked out of the Amalfi, I made sure that my supposed personal concierge knew all of the problems I'd had with my room and the hotel. I was glad to see the Amalfi getting smaller in the mirror as I drove away!"]

In [85]:
vect = cv1.transform(data1).toarray()
print(vect)
print(clf.predict(vect))

[[0 0 0 1 0 0 1 0 2 0 2 0 0 0 0 0 1 0 1 0 2 0 0 0 0 0 0 2 0 1 0 1 0 0 0 0
  0 0 0 0 0 1 0 0 0 3 0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  3 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]
[0]


In [86]:
data2 = ["We will not be back to this hotel. There are several reasons for this, but it began at check-in when the staff were very rude and uptight. Our reservation was lost in the system and they seemed to think it was a strain on their time to have to find it. The room was OK, but the beds were hard and uncomfortable. The room next door had their TV turned up very loudly, and when we called to ask if we could change rooms were were told that was not an option. They didn't offer any other solutions. The food in the restaurants was extremely overpriced and not cooked to our requests. Overall, we wish that we had chosen a different hotel for our stay in Chicago."]

In [87]:
vect = cv1.transform(data2).toarray()
print(vect)
print(clf.predict(vect))

[[0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
  2 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[0]


In [88]:
data3 = ["I have been stayed in Allegro for two days. There are not other big prolems but one thing I'm not satisfied with is the light. It is dark in the room. Also in the bathroom, the light is not good. I have to open curtains. I have complained about this to their reception. They said they will fix it, but I did not get any response. I am very disappointed and I will never come back again"]

In [89]:
vect = cv1.transform(data3).toarray()
print(vect)
print(clf.predict(vect))

[[1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[1]
